In [7]:
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.models import Model
from keras import optimizers, initializers, regularizers, metrics
from tensorflow import keras
from tensorflow.python.keras import layers,Sequential
from tensorflow.keras.layers import Conv2D,Dropout,Dense,Dropout,Activation,MaxPooling2D,Flatten
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow_datasets as tfds
from keras.optimizers import Adam, RMSprop

from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, BatchNormalization
from keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint

In [3]:
!unzip -qq '/content/drive/MyDrive/TRAIN/car.zip' -d '/content/drive/MyDrive/TRAIN/cars'

In [11]:
data_dir = '/content/drive/MyDrive/TRAIN/cars'
prediction_dir = '/content/drive/MyDrive/for prediction/b/for prediction/'

In [9]:
batch_size = 64
img_height = 224
img_width = 224

In [12]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 9900 files belonging to 33 classes.
Using 7920 files for training.


In [13]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 9900 files belonging to 33 classes.
Using 1980 files for validation.


In [14]:
# 프로세스 튜닝
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [15]:
from tensorflow.keras.applications.resnet50 import ResNet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))

for layer in base_model.layers:
  layer.trainable = True

model = Sequential()

model.add(tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(224,224,3)))

model.add(base_model)

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(32, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten()) 

model.add(Dense(512, kernel_regularizer=regularizers.l2(0.0001)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(33, activation='softmax'))

model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy, 
optimizer=RMSprop(learning_rate=0.001), metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 224, 224, 3)       0         
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 conv2d (Conv2D)             (None, 7, 7, 64)          1179712   
                                                                 
 activation (Activation)     (None, 7, 7, 64)          0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 3, 3, 64)         0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 3, 3, 64)          0         
                                                        

In [16]:
hist = model.fit(train_ds,
                 batch_size=64,
                 epochs=30,
                 validation_data = val_ds)

Epoch 1/30
124/124 [==============================] - 133s 695ms/step - loss: 3.5136 - accuracy: 0.0400 - val_loss: 2844.0334 - val_accuracy: 0.0333
Epoch 2/30
124/124 [==============================] - 79s 634ms/step - loss: 3.3860 - accuracy: 0.0553 - val_loss: 4.8486 - val_accuracy: 0.0298
Epoch 3/30
124/124 [==============================] - 80s 643ms/step - loss: 3.5252 - accuracy: 0.0443 - val_loss: 3.5081 - val_accuracy: 0.0263
Epoch 4/30
124/124 [==============================] - 81s 653ms/step - loss: 3.2983 - accuracy: 0.0514 - val_loss: 3.4296 - val_accuracy: 0.0293
Epoch 5/30
124/124 [==============================] - 81s 653ms/step - loss: 3.1815 - accuracy: 0.0644 - val_loss: 3.5070 - val_accuracy: 0.0263
Epoch 6/30
124/124 [==============================] - 81s 654ms/step - loss: 3.0820 - accuracy: 0.0896 - val_loss: 3.5068 - val_accuracy: 0.0263
Epoch 7/30
124/124 [==============================] - 81s 653ms/step - loss: 3.0796 - accuracy: 0.0982 - val_loss: 4.6342 - va

In [ ]:
model.

In [ ]:
model.save('/content/drive/MyDrive/Colab Notebooks/pista1.h5')

NotImplementedError: ignored

In [ ]:
x = model.predict(train_ds)
np.argmax(x,axis=1) # 2
len(x)

1719

In [17]:
model.evaluate(val_ds), model.evaluate(train_ds)

124/124 [==============================] - 22s 178ms/step - loss: 1.7112 - accuracy: 0.4985


([1.825308918952942, 0.46666666865348816],
 [1.711238145828247, 0.49848484992980957])

In [ ]:
model.predict(train_ds)

array([[3.6919899e-03, 9.9630809e-01],
       [3.1940177e-02, 9.6805978e-01],
       [1.0000000e+00, 2.7784602e-14],
       ...,
       [9.9980944e-01, 1.9052681e-04],
       [9.9998140e-01, 1.8558472e-05],
       [5.9063131e-01, 4.0936863e-01]], dtype=float32)

In [ ]:
# 시각화하기
import matplotlib.pyplot as plt

plt.style.use('default')
plt.rcParams['figure.figsize'] = (4, 3)
plt.rcParams['font.size'] = 12

loss = history.history['loss']
plt.plot(loss)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

In [ ]:
# 모델 불러오기
modelx = keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/car_clf_MobileNet.h5')

OSError: ignored